# Task 2. Converting CSV to GeoParquet
## What would be your partition strategy for making the access of the resulting GeoParquet file more efficient?

## Install Python dependencies

In [9]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 12.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 KB 9.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2.1:
      Successfully uninstalled urllib3-2.2.1
Note: you may need to restart the kernel to use updated packages.


## Convert ABS CSV data to Geoparquet file
### Includes:
- Cleaning and renaming columns
- Pivot on the measure data items
- Convert dataframe to Geoparquet and write to a parquet file locally
- Display the output dataframe

In [13]:
import pandas as pd
import geopandas as gpd

df_csv = pd.read_csv('https://api.data.abs.gov.au/files/ABS_ABS_REGIONAL_LGA2021_1.0.0.csv')

# Remove columns not used in target
df_csv.drop(['UNIT_MEASURE: Unit of Measure', 'UNIT_MULT: Unit of Multiplier', 'OBS_STATUS: Observation Status', 'OBS_COMMENT: Observation Comment'], axis=1, inplace=True)

# Clean frequency
df_csv['FREQUENCY'] = df_csv.pop('FREQUENCY: Frequency').str.split(':').str[0]

# Rename time period
df_csv.rename(columns={'TIME_PERIOD: Time Period': 'TIME_PERIOD'}, inplace=True)

# Clean region type
df_csv['REGIONTYPE'] = df_csv.pop('REGIONTYPE: Region Type').str.split(':').str[0]

# Split LGA region into region code and region name
df_csv[['REGION_CODE', 'REGION_NAME']] = df_csv.pop('LGA_2021: Region').str.split(':', n=1, expand=True)

# Replace the measure data item value with a measure name only and use as the column to pivot
df_csv['MEASURE'] = df_csv.pop('MEASURE: Data Item').str.split(':').str[0]

# Create the target table on the measure data item
df_pivot = pd.pivot_table(df_csv, values='OBS_VALUE', index=['DATAFLOW', 'FREQUENCY', 'TIME_PERIOD', 'REGIONTYPE', 'REGION_CODE', 'REGION_NAME'], columns='MEASURE').reset_index()

# Read the geoms data and merge it to the csv dataframe by region code
df_parquet = pd.read_parquet('s3://gbr-dms-data-public/tasks/geoms.parquet')
df = pd.merge(df_pivot, df_parquet, left_on='REGION_CODE', right_on='LGA_CODE21', how='left')

# Remove geoms lga column
df.drop('LGA_CODE21', axis=1, inplace=True)

# Convert and save the dataframe as a Geoparquet file
gdf = gpd.GeoDataFrame(df)
gdf.to_parquet('data.geoparquet')

display(gdf)
gdf.info()


,DATAFLOW,FREQUENCY,TIME_PERIOD,REGIONTYPE,REGION_CODE,REGION_NAME,ACTIV_2,ACTIV_3,ADFS_2,ADFS_3,...,WORK_TRAV_5,WORK_TRAV_6,WORK_TRAV_7,WORK_TRAV_8,WORK_TRAV_9,geometry,minx,miny,maxx,maxy
0,ABS:ABS_REGIONAL_LGA2021(1.0.0),A,2011,LGA2021,10050,Albury,5.0,2383.0,NaN,NaN,...,15860.0,156.0,281.0,90.0,800.0,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\xb6\x0b...,146.81448,-36.11599,147.09921,-35.91389
1,ABS:ABS_REGIONAL_LGA2021(1.0.0),A,2011,LGA2021,10180,Armidale Regional,4.5,1229.0,NaN,NaN,...,7297.0,89.0,114.0,82.0,739.0,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\xa5K\x0...,151.25320,-30.93103,152.42808,-29.93995
2,ABS:ABS_REGIONAL_LGA2021(1.0.0),A,2011,LGA2021,10250,Ballina,5.8,2270.0,NaN,NaN,...,10805.0,105.0,249.0,108.0,677.0,b'\x01\x06\x00\x00\x00\x02\x00\x00\x00\x01\x03...,153.36251,-29.00126,153.60886,-28.70430
3,ABS:ABS_REGIONAL_LGA2021(1.0.0),A,2011,LGA2021,10300,Balranald,4.8,109.0,NaN,NaN,...,556.0,7.0,14.0,16.0,94.0,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\xa2P\x0...,142.45230,-34.97242,144.68920,-33.11571
4,ABS:ABS_REGIONAL_LGA2021(1.0.0),A,2011,LGA2021,10470,Bathurst Regional,4.4,1689.0,NaN,NaN,...,11811.0,96.0,97.0,78.0,829.0,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x95>\x0...,149.11695,-33.99757,149.92217,-32.94862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4350,ABS:ABS_REGIONAL_LGA2021(1.0.0),A,2022,LGA2021,74550,Victoria Daly,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,b'\x01\x06\x00\x00\x00\n\x00\x00\x00\x01\x03\x...,129.00044,-18.68268,132.52288,-13.17573
4351,ABS:ABS_REGIONAL_LGA2021(1.0.0),A,2022,LGA2021,74560,Wagait,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,b'\x01\x06\x00\x00\x00\x02\x00\x00\x00\x01\x03...,130.73207,-12.45712,130.76757,-12.42762
4352,ABS:ABS_REGIONAL_LGA2021(1.0.0),A,2022,LGA2021,74660,West Arnhem,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,b'\x01\x06\x00\x00\x00 \x00\x00\x00\x01\x03\x0...,131.75911,-13.80854,135.00000,-10.90592
4353,ABS:ABS_REGIONAL_LGA2021(1.0.0),A,2022,LGA2021,79399,Unincorporated NT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,b'\x01\x06\x00\x00\x00\x1b\x00\x00\x00\x01\x03...,130.01677,-25.26525,136.79623,-12.00096


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4355 entries, 0 to 4354
Columns: 664 entries, DATAFLOW to maxy
dtypes: float64(657), int64(1), object(6)
memory usage: 22.1+ MB
